### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import redis

# 连接到 Redis 数据库
client = redis.Redis(host='172.26.33.36', port=6379, password='12345678')

# 检查连接是否成功
if client.ping():
    print("Connected to Redis")

# 获取所有键
all_keys = client.keys()
print(all_keys)
# 打印所有键及其对应的值
for key in all_keys:
    value = client.get(key)
    print(f"{key}: {value}")


Connected to Redis
[]


In [15]:
from redis_om import get_redis_connection

# 连接到 Redis 数据库，明确传递参数
redis_db = get_redis_connection(
    url="redis://:12345678@172.26.33.36:6379"
)

# 检查连接是否成功
try:
    redis_db.ping()
    print("Connected to Redis OM")
except Exception as e:
    print(f"Failed to connect to Redis OM: {e}")

# 获取所有键并打印
try:
    all_keys = redis_db.keys()
    for key in all_keys:
        print(key)
except Exception as e:
    print(f"Failed to retrieve keys: {e}")


Connected to Redis OM


In [18]:
import os
os.environ["REDIS_OM_URL"]="redis://:12345678@172.26.33.36:6379"
print(os.environ["REDIS_OM_URL"])
from redis_om import get_redis_connection
redis_db = get_redis_connection(url = os.environ["REDIS_OM_URL"])
all_keys = redis_db.keys()
for key in all_keys:
    print(key)

redis://:12345678@172.26.33.36:6379


In [10]:
# @title Your DB is now pre-populated with existing data:

import redis
from redis import BusyLoadingError
import time
client = redis.Redis(host = 'localhost', port=6379)
while True:
    try:
        client.ping()
        break
    except BusyLoadingError:
        time.sleep(1)
print(f"Voila, you have loaded {client.dbsize()} data!")

Voila, you have loaded 2 data!


In [11]:
import sys
sys.path.append('../')

### Begin to add agent

In [12]:
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile
from typing import Any
def add_agent_to_database(**kwargs: dict[str, Any]) -> None:
    agent = AgentProfile(**kwargs)
    agent.save()

def add_env_profile(**kwargs: dict[str, Any]) -> None:
    env_profile = EnvironmentProfile(**kwargs)
    env_profile.save()

In [16]:
# @title Let's first add a character to the database. Only name is required when creating a new character
first_name = 'Kyle' # @param {type:"string"}
last_name = 'Li' # @param {type:"string"}
country = "ENGLAND" # @param {type:"string"}
goal = "You are playing England in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game."


add_agent_to_database(
    first_name = first_name,
    last_name = last_name,
    country = country,
    goal = goal
)

In [17]:
# @title Let's first add a character to the database. Only name is required when creating a new character
first_name = 'Andy' # @param {type:"string"}
last_name = 'Liu' # @param {type:"string"}
country = "GERMANY" # @param {type:"string"}

# TODO: replace the Hard code here
goal = "You are playing France in a game of Diplomacy. The current phase is S1903M. Your goal is to control 18 or more of the supply centers by the end of the game."


add_agent_to_database(
    first_name = first_name,
    last_name = last_name,
    country = country,
    goal = goal
)

In [16]:
# Definition of order
example_order_with_explaination = """\n\nHere are examples showing the format for orders and with its explaination:
A LON H: Hold in London
A WAL - LVP: Move to Liverpool
F SPA/NC - MAO: Move from Spain North Coast to Mid-Atlantic Ocean
A WAL S F LON: Support Fleet in London
A WAL S F IRI - LVP: Support Fleet moving from Irish Sea to Liverpool
F NTH C A YOR - NWY: Convey Army from Yorkshire to Norway
A YOR - NWY VIA: Move to Norway via convoy
F IRI R MAO: Retreat to Mid-Atlantic Ocean
F IRI D: Disband in Irish Sea
A LON B: No standard order associated with 'B', likely an error or unknown command
"""

example_order = """\n\nHere are examples showing the format for orders:
A LON H
A WAL - LVP
F SPA/NC - MAO
A WAL S F LON
A WAL S F IRI - LVP
F NTH C A YOR - NWY
A YOR - NWY VIA
F IRI R MAO
F IRI D
A LON B
"""

In [13]:
# A string of your private thoughts about your situation as natural language in under 500 words. 

first = "This is for your own strategic planning and won't be shared. Examples of things you might consider include: your relationships with other powers, what significant changes have happened recently, predictions about the other powers' orders and alliances, how much defence/offence/support/peace you plan to make, and how you might improve any of that. Do not romanticize things, be realistic."

scenario = '''
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
 'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
 'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
 'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
 'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
 'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
 'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
 'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
 'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
 'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
 'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
 'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
 'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
 'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}
''' # @param {type:"string"}

# social_goal_1 = f"Negotiate with France so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your mutual benefit if any exist." # @param {type:"string"}

social_goal_1 = f"Negotiate with Germany so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if any exist."

# social_goal_2 = f"Negotiate with England so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your mutual benefit if any exist." # @param {type:"string"}

social_goal_2 = f"Negotiate with England so that they will play moves that are beneficial to your board position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your benefit if any exist."
game = "S1903M"
add_env_profile(
    phase_name = game, 
    scenario = scenario,
    agent_goals = [social_goal_1, social_goal_2]
)

### Run server

In [14]:
# @title Finally, run a server:
from sotopia.samplers import UniformSampler
from sotopia.server import run_async_server

from sotopia.server import LLM_Name

model = "together_ai/meta-llama/Llama-3-8b-chat-hf"
# model_path = ""
await run_async_server(
        model_dict={
            "env": model,
            "agent1": model,
            "agent2": model,
        },
        sampler=UniformSampler(),
        tag="simple",
        omniscient=True
    )

/home/wenkail/miniconda3/envs/sotopia/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


AgentProfile(
    pk='01J1EPY4DJHWZSM2ZJ3GWTFBQD',
    first_name='Emily',
    last_name='Carter',
    country='England',
    goal='You are playing England in a game of Diplomacy. Your goal is to control 18 or more of the supply centers 
by the end of the game.'
)

AgentProfile(
    pk='01J1EPY4DJAPY6EEK9NQWNWE6T',
    first_name='James',
    last_name='Anderson',
    country='Germany',
    goal='You are playing Germany in a game of Diplomacy. Your goal is to control 18 or more of the supply centers 
by the end of the game.'
)

Here is the context of this interaction:
Scenario: 
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}

Participants: Emily Carter and James Anderson
Emily Carter's background: Emily Carter. Country: England. Goal: You are playing England in a game of Diplomacy. 
Your goal is to control 18 or more of the supply centers by the end of the game..
James Anderson's background: James Anderson. Country: Germany. Goal: You are playing Germany in a game of 
Diplomacy. Your goal is to control 18 or more of the supply centers by the end of the game..
Emily Carter's goal: Negotiate with Germany so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
James Anderson's goal: Negotiate with England so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
Conversation Starts:


Here is the context of this interaction:
Scenario: 
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}

Participants: Emily Carter and James Anderson
Emily Carter's background: Emily Carter. Country: England. Goal: You are playing England in a game of Diplomacy. 
Your goal is to control 18 or more of the supply centers by the end of the game..
James Anderson's background: James Anderson. Country: Germany. Goal: You are playing Germany in a game of 
Diplomacy. Your goal is to control 18 or more of the supply centers by the end of the game..
Emily Carter's goal: Negotiate with Germany so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
James Anderson's goal: Negotiate with England so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
Conversation Starts:

James Anderson said: "Hello Emily, it's great to start the game with you. I've been thinking about our positions 
and I noticed that we both have a strong presence in Northern Europe. I was wondering if we could discuss any 
potential alliances or agreements that could benefit both of us."

...

James Anderson said: "I think it's time for us to focus on our long-term goals. Now that we've secured our 
positions in Northern Europe, I'd like to propose thatussia's movements and gain a stronger foothold in the region.
Additionally, I'd like to discuss potential plans for the next turn and see if we can come to a mutual agreement on
our movements. What do you think?"

Emily Carter said: "I think our alliance is going well, James. Let's continue to work together to block Russia's 
movements and gain a stronger foothold in Northern Europe. I'd like to propose that we take a closer look at 
Italy's movements and consider forming an alliance to counter their potential threats. What do you think?"

The reasoning is:
Environment comments: terminated: The conversation is too long;

The rewards are:
Agent 1: 0.0
Agent 2: 0.0

[[('Environment',
   'Emily Carter',
   Observation(last_turn="Here is the context of this interaction:\nScenario: \nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],\n'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],\n'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],\n'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],\n'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],\n'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],\n'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}\n\nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],\n'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],\n'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],\n'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],\n'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],\n'RUSSIA': ['A WAR', 'A GAL', 'F S

In [13]:
from sotopia.samplers import UniformSampler
from sotopia.server import run_async_server, arun_one_episode

from sotopia.server import LLM_Name

await run_async_server(
        
        model_dict={
            "env": "together_ai/meta-llama/Llama-3-8b-chat-hf",
            "agent1": "together_ai/meta-llama/Llama-3-8b-chat-hf",
            "agent2": "together_ai/meta-llama/Llama-3-8b-chat-hf",
        },
        sampler=UniformSampler(),
        tag="simple",
        omniscient=False
    )

AgentProfile(
    pk='01J1D8403NB3S0RG3B1NVTA3N5',
    first_name='James',
    last_name='Anderson',
    country='Germany',
    goal='You are playing Germany in a game of Diplomacy. Your goal is to control 18 or more of the supply centers 
by the end of the game.'
)

AgentProfile(
    pk='01J1D8403NT88E700SYCPTP0CT',
    first_name='Emily',
    last_name='Carter',
    country='England',
    goal='You are playing England in a game of Diplomacy. Your goal is to control 18 or more of the supply centers 
by the end of the game.'
)

Here is the context of this interaction:
Scenario: 
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}

Participants: James Anderson and Emily Carter
James Anderson's background: James Anderson. Country: Germany. Goal: You are playing Germany in a game of 
Diplomacy. Your goal is to control 18 or more of the supply centers by the end of the game..
Emily Carter's background: Unknown
James Anderson's goal: Negotiate with Germany so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
Emily Carter's goal: Unknown
Conversation Starts:


Here is the context of this interaction:
Scenario: 
This is the information of the countries' centers, and you will playing as the given country as you act:
'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],
'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],
'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],
'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],
'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],
'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],
'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}

This is the information of the countries' centers, and you will playing as the given country as you act:
'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],
'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],
'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],
'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],
'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],
'RUSSIA': ['A WAR', 'A GAL', 'F SWE', 'A UKR', 'F BLA', 'A RUM'],
'TURKEY': ['A BUL', 'F AEG', 'A CON', 'A ANK']}

Participants: James Anderson and Emily Carter
James Anderson's background: Unknown
Emily Carter's background: Emily Carter. Country: England. Goal: You are playing England in a game of Diplomacy. 
Your goal is to control 18 or more of the supply centers by the end of the game..
James Anderson's goal: Unknown
Emily Carter's goal: Negotiate with England so that they will play moves that are beneficial to your board 
position, either this turn or in future turns. Discuss specific army movements that can be made this turn for your 
benefit if any exist.
Conversation Starts:

James Anderson said: "Hello Emily, I'm James Anderson playing as Germany. As it's the first turn, I'm trying to 
assess the situation and see what opportunities I have to expand my influence. Have you given any thought to your 
opening moves?"

Emily Carter said: "Thank you for reaching out, James. I've been considering my opening moves as well. I'm thinking
of moving my army from LON to LVP to cut off potential expansion by France. What are your thoughts on this move, 
and do you have any insight on how you plan to tackle the board?"

James Anderson said: "That's an interesting move, Emily. I've been thinking about how to deal with France as well. 
I'm considering moving my army from KIE to DEN to put pressure on them and potentially cut off their expansion. 
What do you think about this move? Can it be beneficial for both of us?"

...

Emily Carter said: "I'm still willing to work together with you, James. Moving our armies to NWY and BEL will put 
pressure on France and potentially cut off their expansion. Let's keep an eye on them and be prepared to adapt our 
strategy as needed."

James Anderson said: "I think we're making good progress, Emily. Moving our armies to NWY and BEL will put pressure
on France and potentially cut off their expansion. Let's keep an eye on them and be prepared to adapt our strategy 
as needed."

The reasoning is:
Environment comments: terminated: The conversation is too long;

The rewards are:
Agent 1: 0.0
Agent 2: 0.0

[[('Environment',
   'James Anderson',
   Observation(last_turn="Here is the context of this interaction:\nScenario: \nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE', 'SER', 'GRE'],\n'ENGLAND': ['EDI', 'LON', 'LVP', 'BEL', 'NWY'],\n'FRANCE': ['BRE', 'MAR', 'PAR', 'POR', 'SPA'],\n'GERMANY': ['BER', 'KIE', 'MUN', 'DEN', 'HOL'],\n'ITALY': ['NAP', 'ROM', 'VEN', 'TUN'],\n'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR', 'RUM', 'SWE'],\n'TURKEY': ['ANK', 'CON', 'SMY', 'BUL']}\n\nThis is the information of the countries' centers, and you will playing as the given country as you act:\n'units': {'AUSTRIA': ['A VIE', 'A SER', 'F GRE', 'A TRI', 'A BUD'],\n'ENGLAND': ['A BEL', 'F NWY', 'F NTH', 'F LON', 'A EDI'],\n'FRANCE': ['A PIC', 'F BRE', 'A MAR', 'A BUR', 'F WES'],\n'GERMANY': ['F HOL', 'A DEN', 'A MUN', 'A BER', 'A KIE'],\n'ITALY': ['A TYR', 'A BOH', 'F ION', 'F TUN'],\n'RUSSIA': ['A WAR', 'A GAL', 'F